In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
import numpy as np
from src.helper.visualization import visualize_mesh, visualize_meshes
import trimesh

import torch
from tqdm import tqdm

import numpy as np
import os
import h5py
from src.helper.paths import BASELINES_PATH
from src.helper.preprocessing import get_pc_distances
from src.partglot.utils.processing import vstack2dim
from src.partglot.utils.partglot_bspnet_preprocess import normalize_pointcloud, rotate_pointcloud
from src.helper.visualization import visualize_pointclouds_parts_partglot

root = pathlib.Path("/mnt/hdd/shapenetcore_partanno_segmentation_benchmark_v0/03001627/")
shapenet_root = pathlib.Path("/mnt/hdd/ShapeNetCore.v2/03001627/")


data_dir = BASELINES_PATH / 'PartGlot/data'

h5_data = h5py.File(data_dir / "shapenet_partseg_chair_bsp.h5")
segs_data = h5_data["data"][:].astype(np.float32)

In [3]:
# selects cherry picked supersegments from PartGlot
# to extract the corresponding Shappe Net meshes
f_segs_data = [pc for i, pc in enumerate(segs_data) if i in (40, 55, 70)] 

In [4]:
# loads all shapenet points and lables
points = []
labels = []
meshes = []

for i, point_path in tqdm(list(enumerate(root.joinpath("points").iterdir()))):
    
    points.append(np.loadtxt(point_path))
    labels.append(np.loadtxt(root.joinpath("points_label").joinpath(point_path.stem).with_suffix(".seg"), dtype=int))


100%|██████████| 3746/3746 [00:31<00:00, 120.07it/s]


In [6]:
# normalizes all point clouds from ShapeNet
prepros_sn_pc = [normalize_pointcloud(rotate_pointcloud(vstack2dim(pc)))['pc'] for pc in points]

# gets point clouds with smallest distance
# from the ShapeNet w.r.t. to each of the cherry picked
# super-segments from PartGlot
indices = []
all_dists = []
for src_pc in tqdm(f_segs_data):
    n_src = normalize_pointcloud(vstack2dim(src_pc))['pc']
    pc_dists = get_pc_distances(n_src, prepros_sn_pc)
    closest_pc = min(pc_dists, key = lambda t: t[1])
    print(closest_pc)
    indices.append(closest_pc[0])
    all_dists.append(pc_dists)

3746it [00:12, 299.80it/s]0<?, ?it/s]
 33%|███▎      | 1/3 [00:12<00:24, 12.50s/it]

(1193, 2.31311297416687)


3746it [00:11, 326.56it/s]
 67%|██████▋   | 2/3 [00:23<00:11, 11.90s/it]

(2019, 11.044352531433105)


3746it [00:11, 326.47it/s]
100%|██████████| 3/3 [00:35<00:00, 11.82s/it]

(134, 7.2849836349487305)


In [10]:
# visualize results to check if they are correct
# plots the overlapping point clouds of the super-segments
# and the most similar ShapeNet point cloud
# hint: tweak variable `i` to check every point cloud
i = 0
n_src = normalize_pointcloud(vstack2dim(f_segs_data[i]))['pc']
ref = prepros_sn_pc[indices[i]]
visualize_pointclouds_parts_partglot([n_src, ref])

Visualization rendering started...


Output()

In [11]:
# load meshes from the found indices
# prints the indices and paths of the corresponding matches
for i, point_path in tqdm(list(enumerate(root.joinpath("points").iterdir()))):
    if i not in indices:
        continue
    points.append(np.loadtxt(point_path))
    labels.append(np.loadtxt(root.joinpath("points_label").joinpath(point_path.stem).with_suffix(".seg"), dtype=int))
    with open(shapenet_root.joinpath(point_path.stem).joinpath("models").joinpath("model_normalized.obj")) as fp:
        meshes.append(trimesh.exchange.obj.load_obj(fp, include_color=False, include_texture=False))
    print(f"i = {i}")
    print(f"point cloud path = {point_path}")
    print(f'labels path = {root.joinpath("points_label").joinpath(point_path.stem).with_suffix(".seg")}')
    print(f'mesh path = {shapenet_root.joinpath(point_path.stem).joinpath("models").joinpath("model_normalized.obj")}')
    print(20*"=")

100%|██████████| 3746/3746 [00:00<00:00, 24191.60it/s]

i = 134
point cloud path = /mnt/hdd/shapenetcore_partanno_segmentation_benchmark_v0/03001627/points/47eff1e5c09cec7435836c728d324152.pts
labels path = /mnt/hdd/shapenetcore_partanno_segmentation_benchmark_v0/03001627/points_label/47eff1e5c09cec7435836c728d324152.seg
mesh path = /mnt/hdd/ShapeNetCore.v2/03001627/47eff1e5c09cec7435836c728d324152/models/model_normalized.obj
i = 1193
point cloud path = /mnt/hdd/shapenetcore_partanno_segmentation_benchmark_v0/03001627/points/88c39cf1485b497bfbb8cbddab1c2002.pts
labels path = /mnt/hdd/shapenetcore_partanno_segmentation_benchmark_v0/03001627/points_label/88c39cf1485b497bfbb8cbddab1c2002.seg
mesh path = /mnt/hdd/ShapeNetCore.v2/03001627/88c39cf1485b497bfbb8cbddab1c2002/models/model_normalized.obj
i = 2019
point cloud path = /mnt/hdd/shapenetcore_partanno_segmentation_benchmark_v0/03001627/points/c1b312919af633f8f51f77a6d7299806.pts
labels path = /mnt/hdd/shapenetcore_partanno_segmentation_benchmark_v0/03001627/points_label/c1b312919af633f8f51f

In [14]:
# plots meshes to confirm consistency
# between point clouds and shapenet meshes
i = 0
p_meshes = []
for g in meshes[i]['geometry'].values():
    faces, vertices = g['faces'], g['vertices']
    p_meshes.append({'faces': faces, 'vertices': vertices})
    visualize_meshes(p_meshes)

Output()

Output()

Output()

Output()